In [1]:
import xarray as xr
import numpy as np
import os
from glob import glob
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
dir='/nobackupp27/dmenemen/public/geos_ecco/c1440_llc2160/holding/'

In [3]:
odir='/nobackupp27/afahad/project/subgrid_tenden/data/'

In [4]:
os.chdir(dir+'inst_01hr_3d_T_Mv/')

In [5]:
files=sorted(glob('*.nc4'))
files=files[2:][::6]

In [6]:
files[:10]

['DYAMOND_c1440_llc2160.inst_01hr_3d_T_Mv.20200120_0000z.nc4',
 'DYAMOND_c1440_llc2160.inst_01hr_3d_T_Mv.20200120_0600z.nc4',
 'DYAMOND_c1440_llc2160.inst_01hr_3d_T_Mv.20200120_1200z.nc4',
 'DYAMOND_c1440_llc2160.inst_01hr_3d_T_Mv.20200120_1800z.nc4',
 'DYAMOND_c1440_llc2160.inst_01hr_3d_T_Mv.20200121_0000z.nc4',
 'DYAMOND_c1440_llc2160.inst_01hr_3d_T_Mv.20200121_0600z.nc4',
 'DYAMOND_c1440_llc2160.inst_01hr_3d_T_Mv.20200121_1200z.nc4',
 'DYAMOND_c1440_llc2160.inst_01hr_3d_T_Mv.20200121_1800z.nc4',
 'DYAMOND_c1440_llc2160.inst_01hr_3d_T_Mv.20200122_0000z.nc4',
 'DYAMOND_c1440_llc2160.inst_01hr_3d_T_Mv.20200122_0600z.nc4']

In [7]:
C90=xr.open_dataset('/nobackupp27/afahad/exp/GMC90_1bugfx/moist_internal_rst',decode_times=False)

In [8]:
area=xr.open_dataset(dir+'const_2d_asm_Mx/DYAMOND_c1440_llc2160.const_2d_asm_Mx.20200122_0000z.nc4')[['AREA']]

In [9]:
def coarsen_dataarray(da_in, area,  c5760_res=None,lon_c90=None, lat_c90=None):
    """
    Coarsen a cubed-sphere DataArray from C5760 to C90 with area-weighted averaging.
    
    Parameters:
    -----------
    da_in : xr.DataArray
        Input DataArray to be coarsened. Last two dimensions should be the spatial dimensions.
    area : xr.DataArray or np.ndarray
        Area weights array (should be the same shape as the spatial dimensions of da_in)
    lon_c90 : array-like, optional
        Longitude coordinates for the C90 grid. If None, will be created as a simple range.
    lat_c90 : array-like, optional
        Latitude coordinates for the C90 grid. If None, will be created as a simple range.
        
    Returns:
    --------
    xr.DataArray
        Coarsened DataArray at C90 resolution
    """
    # Define resolutions
    c5760_res = 1440
    c90_res = 90
    factor = c5760_res // c90_res  # 64

    original_dims = list(da_in.dims)
    odims=list(da_in.dims)[:2]+['face', 'y', 'x']
    
    # Create default coordinates if not provided
    if lon_c90 is None:
        lon_c90 = np.arange(c90_res)
    if lat_c90 is None:
        lat_c90 = np.arange(6 * c90_res)  # 540 points across all 6 faces
    
    # Get original dimensions excluding the last two (spatial dimensions)
    
    
    # Convert area to numpy if it's a DataArray
    if isinstance(area, xr.DataArray):
        area_data = area.values
    else:
        area_data = area
    
    # Reshape area to (6, 5760, 5760)
    area_faces = area_data.reshape(6, c5760_res, c5760_res)
    area_da = xr.DataArray(area_faces, dims=['face', 'y', 'x'])
    
    # Precompute coarsened areas
    coarsened_area = area_da.coarsen(y=factor, x=factor, boundary='trim').sum()
    
    # Convert to 6*c90_res, c90_res shape for final output
    area_c90 = coarsened_area.values.reshape(6 * c90_res, c90_res)
    
    # Reshape data to separate faces (preserve other dimensions)
    if len(da_in.shape) > 3:
        # Handle case with additional dimensions
        #data_reshaped = da_in.values.reshape(da_in.shape[:-3] + (6, c5760_res, c5760_res))
        data_reshaped = da_in.values.reshape( da_in.shape[:-2] + (6, c5760_res, c5760_res) )
    else:
        # Handle case with only spatial dimensions
        data_reshaped = da_in.values.reshape(6, c5760_res, c5760_res)
    
    # Create temporary DataArray with face/y/x dims
    if len(da_in.shape) > 3:
        da_faces = xr.DataArray(
            data_reshaped,
            dims=original_dims[:-2] + ['face', 'y', 'x'],
            attrs=da_in.attrs
        )
    else:
        da_faces = xr.DataArray(
            data_reshaped,
            dims=['face', 'y', 'x'],
            attrs=da_in.attrs
        )
    
    # Area-weighted averaging
    numerator = (da_faces * area_da).coarsen(y=factor, x=factor, boundary='trim').sum()
    denominator = coarsened_area
    result = (numerator / denominator).values
    
    # Reshape to final C90 dimensions
    if len(da_in.shape) > 3:
        #result_reshaped = result.reshape(da_in.shape[:-3] + (6,  c90_res, c90_res))
        result_reshaped = result.reshape( da_in.shape[:-2] + (6, c90_res, c90_res) )
    else:
        result_reshaped = result.reshape(6,  c90_res, c90_res)

    # print(result_reshaped.shape)
    # print(original_dims)
    # Build new DataArray
    #print(odims, result_reshaped.shape)
    return xr.DataArray(
        result_reshaped,
        dims=odims,
        # coords={'lat': lat_c90, 'lon': lon_c90},
        attrs=da_in.attrs
    )


In [10]:
files=files[122+367:]

In [11]:
import dask

In [12]:
# create Hres dprdt
for i in tqdm(range(len(files))):
            ds1=xr.open_mfdataset(files[i])#.QV#.sel(lev=int(l))
            ds2=xr.open_mfdataset(files[i+1])#.QV.sel(lev=int(l))
            dT=ds2.T.data-ds1.T.compute()
            dT=coarsen_dataarray(dT,area.AREA)
            dT.rename('dT').to_netcdf(odir+'dT_'+files[i])

100%|████████████████████████████████████████████████████████████▉| 1238/1239 [4:49:39<00:14, 14.04s/it]


IndexError: list index out of range

In [ ]:
# # create C90 of pr
# #files=files[383:]
# for i in tqdm(range(len(files))):
#     ds1=xr.open_dataset(files[i])
#     pr=ds1.T
#     pr=coarsen_dataarray(pr,area.AREA)#,lon_c90=C90.lon,lat_c90=C90.lat)
#     pr.rename('T').to_netcdf(odir+'C90T_'+files[i])

In [ ]:
# PS inst_15mn_2d_asm_Mx
#